<a href="https://colab.research.google.com/github/gnishtha/3/blob/master/Briq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip '/content/drive/My Drive/briq.zip'

Archive:  /content/drive/My Drive/briq.zip
   creating: briq/
   creating: briq/pdf_test/
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_0.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_1.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_2.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_3.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_4.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_5.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_6.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_7.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_8.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_9.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_10.jpg  
  inflating: briq/pdf_test/Ferguson Batch from Nicki 2020-03-30.pdf_11.jp

In [12]:
!pip install google-cloud-vision

     |████████████████████████████████| 440kB 3.4MB/s 


In [9]:
!pip install ruamel.yaml

     |████████████████████████████████| 112kB 3.5MB/s 
     |████████████████████████████████| 552kB 6.6MB/s 


In [11]:
import sys
import json
from collections.abc import Mapping, Sequence
from collections import OrderedDict
import ruamel.yaml

# if you instantiate a YAML instance as yaml, you have to explicitly import the error
from ruamel.yaml.error import YAMLError


yaml = ruamel.yaml.YAML()  # this uses the new API
# if you have standard indentation, no need to use the following
yaml.indent(sequence=4, offset=2)

input_file = '/content/briq/Mcmaster/config.yaml'
intermediate_file = '/content/briq/Mcmaster/config.json'
#output_file = 'output.yaml'


class OrderlyJSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Mapping):
            return OrderedDict(o)
        elif isinstance(o, Sequence):
            return list(o)
        return json.JSONEncoder.default(self, o)


def yaml_2_json(in_file, out_file):
    with open(in_file, 'r') as stream:
        try:
            datamap = yaml.load(stream)
            with open(out_file, 'w') as output:
                output.write(OrderlyJSONEncoder(indent=2).encode(datamap))
        except YAMLError as exc:
            print(exc)
            return False
    return True


yaml_2_json(input_file, intermediate_file)
with open(intermediate_file) as fp:
    sys.stdout.write(fp.read())

{
  "Invoice Number": {
    "x0": 1120,
    "x1": 1620,
    "y0": 370,
    "y1": 425,
    "type": "int"
  },
  "Invoive Date": {
    "x0": 1200,
    "x1": 1620,
    "y0": 425,
    "y1": 480,
    "type": "date"
  },
  "total amount": {
    "x0": 1120,
    "y0": 1620,
    "x1": 320,
    "y1": 270,
    "type": "float"
  },
  "table": {
    "x0": 20,
    "y0": 900,
    "x1": 1650,
    "end": "merchandise",
    "line": {
      "x0": 70,
      "x1": 300,
      "type": "str"
    },
    "Product": {
      "x0": 300,
      "x1": 970,
      "type": "str"
    },
    "ordered": {
      "x0": 970,
      "x1": 1100,
      "type": "str"
    },
    "shiped": {
      "x0": 1100,
      "x1": 1230,
      "type": "int"
    },
    "balance": {
      "x0": 1230,
      "x1": 1360,
      "type": "str"
    },
    "Price": {
      "x0": 1360,
      "x1": 1500,
      "type": "str"
    },
    "Total": {
      "x0": 1500,
      "x1": 1640,
      "type": "float"
    }
  },
  "moveable_data": {
    "subtotal": {
   

In [0]:
  import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/briq/briq-chronicle-production-394361177ce1.json"

In [0]:
import io
import os
from google.cloud import vision
from google.cloud.vision import types
import pandas as pd
import json
import cv2
import matplotlib.pyplot as plt
from collections import Counter
from dateutil import parser
import re
from google.cloud import storage
from google.protobuf import json_format

In [0]:
def detect_text_v1(path):
    """Detects text in the file. Using google vision api (output will be in dataframe format)"""
    print(path)
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    txt = []
    x0 = []
    x1 = []
    y0 = []
    y1 = []
    centerx = []
    centery = []
    for text in texts[1:]:
        #print('\n"{}"'.format(text.description))
        txt.append(text.description)
        vertices = ([(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])
        x0.append(vertices[0][0]) 
        x1.append(vertices[1][0])
        y0.append(vertices[0][1])
        y1.append(vertices[2][1])
        centerx.append(int((vertices[0][0]+vertices[1][0])/2))
        centery.append(int((vertices[0][1]+vertices[2][1])/2))
        #print('bounds: ',(vertices))
    return pd.DataFrame({"text":txt,"x0":x0,"y0":y0,"x1":x1,"y1":y1,"centerx":centerx,"centery":centery})
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [0]:
"""
input -> start_text of and end_text of field
output -> return info of field (for moveable data)
"""
def get_moveable_field_data(data,key,field):
    start_data = data[data['text'].str.contains(key['start'], case=False)]
    end_data = data[data['text'].str.contains(key['end'], case=False)]
    temp_dct = {}
    for ind, row in start_data.iterrows():
        temp_dct[str(ind)+"_"+str(abs(end_data['line'] - row['line']).idxmin())] = min(abs(end_data['line'] - row['line']))
    indxs = min(temp_dct, key=temp_dct.get)
    strt = int(indxs.split("_")[0])
    end = int(indxs.split("_")[1])
    end_line = data.loc[end,'line']
    strt_line = data.loc[strt,'line']
    if field != "table":
        if end_line != strt_line:
            return data_format((" ").join(data[(data.loc[strt,'x1'] < data['x0']) & (strt_line == data['line'])]['text']),key['type'])
        else:

          return data_format((" ").join(data[(data.loc[strt,'x1'] < data['x0']) & (data.loc[end,'x0'] > data['x1'])  & (strt_line == data['line'])]['text']),key["type"])
    else:
        #print(key)
        return get_table_structure_data(data[(end_line > data['line']) & (strt_line <= data['line'])],key,True)

In [0]:
"""
Convert the types of info field as given in yaml file
"""
def data_format(x,frmt):
    try:
        if frmt == "float":
            return float(x)
        elif frmt == "int":
            return int(x) 
        elif frmt == 'date':
            return parser.parse(x).isoformat()
        else:
            return str(x)  
    except:
        return str(x)

In [0]:
"""
return key which has particular value (used in Pipline_for_get_complete_row function)
"""
def get_key(my_dict,val): 
    for key, value in my_dict.items(): 
        if val == value: 
            return key 

In [0]:
"""
This function is use to convert fetched table as given in images 
"""
def get_complete_row(df,key):
    restructure_data = {new_list: [] for new_list in df.columns}
    for index,row in df.iterrows():
        try:
            if row[key] != "":
                for i in df.columns:
                    restructure_data[i].append(row[i])
            else:
                for i in df.columns:
                    restructure_data[i] = restructure_data[i][:-1] + [restructure_data[i][-1] + row[i]]
        except:
            pass
    return restructure_data

In [0]:
"""
input -> raw table data (unformatted)
output -> same format tabe as in image
function use-> get_key(), get_complete_row()
"""
def Pipline_for_get_complete_row(table_data):
    df = pd.DataFrame(table_data)
    tmp = {new_list: len(list(filter(lambda x: x != "", df[new_list]))) for new_list in df.columns}
    val = Counter(list(tmp.values())).most_common(1)[0][0]
    key = get_key(tmp,val)
    updated_table_data = get_complete_row(df,key)
    return updated_table_data

In [0]:
"""
This function takes google vision data, and extract the table data (unformatted)
"""
def get_table_structure_data(data,key,moveable_flag = False):
    if moveable_flag == True:
        df = data
    else:
        index = data.loc[data[data['text'].str.contains(key['end'], case=False) & (data['y0'] > key['y0'])].index.values[0],'line']
        df = data[(key['x0'] < data['centerx']) & (data['centerx'] < key['x1']) & (key['y0'] < data['centery']) & (data['line'] < index)]
    #print(df,"-->",data[data['text'].str.contains(key['end'], case=False) & (data['y0'] > key['y0'])],index)
    key_list = list(set(list(key.keys()))-set(['x0','x1','y0','end','start']))
    table_data = {new_list: [] for new_list in key_list} 
    for i in key_list:
        flag = False
        for k in list(set(list(df['line']))):
            temp_df = df[(df['line']==k) & (key[i]['x0'] < df['centerx']) & (key[i]['x1'] > df['centerx'])]
            if flag:
                #print(key[i]["type"])
                table_data[i].append((" ").join(temp_df['text']))
            flag = True
    structure_table = Pipline_for_get_complete_row(table_data)
    type_mapping = {i:key[i]['type'] for i in key.keys() if type(key[i]) == dict}
    updated_structure_table = {i:[data_format(x,type_mapping[i]) for x in structure_table[i]] for i in structure_table.keys()}
    return updated_structure_table

In [0]:
"""
input -> folder path in which images and config yaml file is stored
output-> will extract all info's of field that are mention in yaml file
"""
def get_invoice_data_v1(path):
    files = list(set(os.listdir(path))-set(['config.json']))
    print(files)
    with open(os.path.join(path,'config.json'),'r') as f:
        mapping = json.load(f)
    data = []
    t=[]
    for file in files:
        temp_df = detect_text_v1(os.path.join(path,file))
        tmp_dict = {"file":file}
        for k in mapping.keys():
            if k == 'table':
                tmp_dict[k] = get_table_structure_data(get_sequence_data(temp_df),mapping[k])
                print(mapping[k])
            elif k == "moveable_data":
                keys = mapping[k]
                for key in keys:
                    tmp_dict[key] = get_moveable_field_data(get_sequence_data(temp_df),keys[key],key)
            else:
                filter_data = temp_df[(mapping[k]['x0'] < temp_df['centerx']) & (temp_df['centerx'] < mapping[k]['x1']) & (mapping[k]['y0'] < temp_df['centery']) & (temp_df['centery'] < mapping[k]['y1'])].sort_values(by='x0')
                tmp_dict[k] = data_format((" ").join(get_sequence_data(filter_data)['text']),mapping[k]["type"])
        data.append(tmp_dict)
    return data

In [39]:

get_invoice_data_v1("/content/briq/Mcmaster/")

['Invoice 38039914 for PO 4520-1.jpg', 'Invoice 38105146 for PO 220020-1.jpg']
/content/briq/Mcmaster/Invoice 38039914 for PO 4520-1.jpg
/content/briq/Mcmaster/Invoice 38105146 for PO 220020-1.jpg


[{'Invoice Number': 38039914,
  'Invoive Date': '2020-04-13T00:00:00',
  'file': 'Invoice 38039914 for PO 4520-1.jpg',
  'subtotal': 23.56,
  'table': {'Price': ['5.89Еach'],
   'Product': ['Oven Cleaner, 18 oz. Aerosol Can'],
   'Total': [23.56],
   'balance': [''],
   'line': ['1 7141T41'],
   'ordered': ['4Еach'],
   'shiped': [4]},
  'tax': 1.88,
  'total amount': ''},
 {'Invoice Number': 38105146,
  'Invoive Date': '2020-04-14T00:00:00',
  'file': 'Invoice 38105146 for PO 220020-1.jpg',
  'subtotal': 485.02,
  'table': {'Price': ['34.18Per Pack', '7.68Еach', '74.31Each'],
   'Product': ['Trade No. 18 Precision Knife Blades, Packs of 100',
    'Easy-Grip Precision Knife',
    'Forged Steel Hoist Ring for Lifting, 3/4"-10 ThreadSize, 6-7/8" Overall Height'],
   'Total': [34.18, 153.6, 297.24],
   'balance': ['', '', ''],
   'line': ['1 38995A69', '35515A12', '2994T71'],
   'ordered': ['1Рack', '20Each', '4Еach'],
   'shiped': [1, 20, 4]},
  'tax': 38.8,
  'total amount': ''}]

In [28]:
!ls '/content/briq/Mcmaster'

 config.json			      'Invoice 38105146 for PO 220020-1.jpg'
'Invoice 38039914 for PO 4520-1.jpg'


In [0]:
!cd '/content/briq/Mcmaster' && rm -R '.ipynb_checkpoints'